## Mapping historic neighborhoods in Lisbon
### Digital sketches and online geo-tagged activity

#### Urban places and regions in GIScience – concepts, methods and challenges
AGILE 2023, Delft, Netherlands

CityMe project - https://cityme.novaims.unl.pt/ 

FCT - Portuguese national funding agency for science, research and technology EXPL/GES-URB/1429/2021

<img src=https://zap.aeiou.pt/wp-content/uploads/2015/07/5a716b2f095a5b4f0b198a53ac7cbef4.jpeg width="400">

#### We are usingg data collected from our survey as well as geo-tagged user-generated content collected for the city of Lisbon
We will use these data sources to explore the perceived extents of the famous historic neighborhoods of Alfama, Mouraria and Bairro Alto. These neighborhoods are part of Lisbon's history, identity and culture, but as any vernacular historic neighborhoods, boundaries might change according to individuals, mapping purposes as well as local communities.

Our aim is to use spatial representations and visualizations that might help us in having a better spatial understanding regarding the neighborhoods, their cores, how their boundaries differ and the differences not only between digital surveyed data and online sources, but also between different online sources

Let's start by importing our required dependencies for the exercise

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
from shapely.wkt import loads
from shapely.geometry import Polygon
from scipy.spatial import distance
import contextily as cx
from contextily.tile import warp_img_transform, warp_tiles, _warper
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib_scalebar.scalebar import ScaleBar
from libpysal.cg.alpha_shapes import alpha_shape_auto
from esda.adbscan import ADBSCAN, get_cluster_boundary, remap_lbls
import numba
from shapely.ops import nearest_points

#### Now let's import the data.

The UGC data consists of csv. files of retrieved (APIs and online resources) activity from Flickr, AirBnB, Twitter and Instagram with coordinates. 

The survey data correspond to shapefiles for each historic neighborhood containing the collected digital sketches

Additionally, we will also need to upload Lisbon's city boundaries to filter out instances that are outside the city.

Here, we upload the data, make sure that they all have the same UTM projection and transform csv. data into geodataframes with the corresponding lat/lon information.

In [ ]:
#city boundaries shapefile
lisbon = gpd.read_file('./shapefiles/Limite_Cartografia.shp')
lisbon = lisbon.to_crs('EPSG:27429')

#survey_responses shapefiles
alfama_survey = gpd.read_file('./shapefiles/alfama_survey.shp')
mouraria_survey = gpd.read_file('./shapefiles/mouraria_survey.shp')
bairro_survey = gpd.read_file('./shapefiles/bairro_survey.shp')

#online sources csv. files
twitter = pd.read_csv("twitter.csv")
instagram = pd.read_csv("instagram.csv", low_memory=False)
flickr = pd.read_csv("flickr.csv")
airbnb = pd.read_csv("airbnb.csv")

#turning pandas dataframes into geopandas geodataframes
#Twitter
twitter['geometry_real'] = twitter['geometry'].apply(lambda x: loads(x))
twitter_gdf = gpd.GeoDataFrame(twitter, geometry= twitter.geometry_real, crs='EPSG:4326')
twitter_gdf = twitter_gdf.to_crs('EPSG:27429')

#Instagram
instagram_gdf = gpd.GeoDataFrame(instagram, geometry=gpd.points_from_xy(instagram.lng, instagram.lat), crs='EPSG:4326')
instagram_gdf = instagram_gdf.to_crs('EPSG:27429')

#Flickr
flickr_gdf = gpd.GeoDataFrame(flickr, geometry=gpd.points_from_xy(flickr.lng, flickr.lat), crs='EPSG:4326')
flickr_gdf = flickr_gdf.to_crs('EPSG:27429')

#Airbnb
airbnb_gdf = gpd.GeoDataFrame(airbnb, geometry=gpd.points_from_xy(airbnb.longitude, airbnb.latitude), crs='EPSG:4326')
airbnb_gdf = airbnb_gdf.to_crs('EPSG:27429')

##### Now let's clip the online sources to Lisbon's boundaries and take a better look at them

In [ ]:
#clipping sources to Lisbon's city boundaries using geopandas clip
twitter = gpd.clip(twitter_gdf, lisbon.geometry)
instagram = gpd.clip(instagram_gdf, lisbon.geometry)
flickr = gpd.clip(flickr_gdf, lisbon.geometry)
airbnb = gpd.clip(airbnb_gdf, lisbon.geometry)

In [ ]:
#Taking a peek at the geodataframes and their size
#Twitter
print(twitter.shape[0])
twitter.head(5)

In [ ]:
#Instagram
print(instagram.shape[0])
instagram.head(5)

In [ ]:
#Flickr
print(flickr.shape[0])
flickr.head(5)

In [ ]:
#Airbnb
print(airbnb.shape[0])
airbnb.head(50)

##### Now let's check their unfiltered distribution onto the basemap of Lisbon

Feel free to change colors, transparencies, basemap, markersizes or any other visualization feature throughout the notebook

In [ ]:
#Plotting all UGC data 

#Figure size
plt.rcParams['figure.figsize'] = [6, 6] 

#Subplots
fig, myax = plt.subplots()

#Plotting and defining markersize, color and transparency
flickr.plot(ax=myax,markersize=10, color='mediumpurple', alpha=0.6)
airbnb.plot(ax=myax,markersize=10, color='plum', alpha=0.6)
twitter.plot(ax=myax,markersize=10, color='steelblue', alpha=0.6)
instagram.plot(ax=myax,markersize=10, color='firebrick', alpha=0.6)

#Adding base-map with contextily
cx.add_basemap(myax, crs=twitter.crs.to_string(), source=cx.providers.CartoDB.Positron, zoom=15,attribution_size=0)

#Removing axis, adding scale bar and title
myax.axis("off")
myax.add_artist(ScaleBar(1, location='upper left'))
myax.set_title('UGC')

#Defining personalized legends
twitter_leg = plt.scatter([],[], marker='o',color='steelblue', label='Twitter')
instagram_leg = plt.scatter([],[], marker='o',color='firebrick', label='Instagram')
flickr_leg = plt.scatter([],[], marker='o',color='mediumpurple', label='Flickr')
airbnb_leg = plt.scatter([],[], marker='o',color='plum', label='AirBnB')

#Adding legend
plt.legend(handles=[twitter_leg, instagram_leg, flickr_leg, airbnb_leg], loc='lower right')

#### Now we are going to filter instances where there are mentions to the neighborhoods
Columns with textual data generated/filled by users and metadata for each source and neighborhood

##### While Twitter, Instagram and Flickr we are using more fields, we opt to only use the name field in Airbnb, why is that?

In [ ]:
#Alfama
#Twitter and columns
twitter_alfama = twitter[twitter['text'].str.contains("Alfama|alfama") | twitter['full_name'].str.contains("Alfama|alfama") | twitter['name'].str.contains("Alfama|alfama")]

#Instagram and columns
instagram_alfama = instagram[instagram['caption_text'].str.contains("Alfama|alfama") | instagram['name'].str.contains("Alfama|alfama") | instagram['address'].str.contains("Alfama|alfama")]

#Flickr and columns
flickr_alfama = flickr[flickr['title'].str.contains("Alfama|alfama") | flickr['tags'].str.contains("Alfama|alfama") | flickr['neighbourhood'].str.contains("Alfama|alfama") | flickr['locality'].str.contains("Alfama|alfama") | flickr['desc'].str.contains("Alfama|alfama") | flickr['county'].str.contains("Alfama|alfama") | flickr['region'].str.contains("Alfama|alfama")]

#Airbnb and columns
airbnb_alfama = airbnb[airbnb['name'].str.contains("Alfama|alfama", na=False)]

In [ ]:
#Mouraria
#Twitter and columns
twitter_mouraria = twitter[twitter['text'].str.contains("Mouraria|mouraria") | twitter['full_name'].str.contains("Mouraria|mouraria") | twitter['name'].str.contains("Mouraria|mouraria")]

#Instagram and columns
instagram_mouraria = instagram[instagram['caption_text'].str.contains("Mouraria|mouraria") | instagram['name'].str.contains("Mouraria|mouraria") | instagram['address'].str.contains("Mouraria|mouraria")]

#Flickr and columns
flickr_mouraria = flickr[flickr['title'].str.contains("Mouraria|mouraria") | flickr['tags'].str.contains("Mouraria|mouraria") | flickr['neighbourhood'].str.contains("Mouraria|mouraria") | flickr['locality'].str.contains("Mouraria|mouraria") | flickr['desc'].str.contains("Mouraria|mouraria") | flickr['county'].str.contains("Mouraria|mouraria") | flickr['region'].str.contains("Mouraria|mouraria")]

#Airbnb and columns
airbnb_mouraria = airbnb[airbnb['name'].str.contains("Mouraria|mouraria", na=False)]

In [ ]:
#Bairro Alto
#Twitter and columns
twitter_bairro = twitter[twitter['text'].str.contains("Bairro Alto|bairro alto") | twitter['full_name'].str.contains("Bairro Alto|bairro alto") | twitter['name'].str.contains("Bairro Alto|bairro alto")]

#Instagram and columns
instagram_bairro = instagram[instagram['caption_text'].str.contains("Bairro Alto|bairro alto") | instagram['name'].str.contains("Bairro Alto|bairro alto") | instagram['address'].str.contains("Bairro Alto|bairro alto")]

#Flickr and columns
flickr_bairro = flickr[flickr['title'].str.contains("Bairro Alto|bairro alto") | flickr['tags'].str.contains("Bairro Alto|bairro alto") | flickr['neighbourhood'].str.contains("Bairro Alto|bairro alto") | flickr['locality'].str.contains("Bairro Alto|bairro alto") | flickr['desc'].str.contains("Bairro Alto|bairro alto") | flickr['county'].str.contains("Bairro Alto|bairro alto") | flickr['region'].str.contains("Bairro Alto|bairro alto")]

#Airbnb and columns
airbnb_bairro = airbnb[airbnb['name'].str.contains("Bairro Alto|bairro alto", na=False)]

#### Let's check the distribution of UGC data mentions to neighborhoods

In [ ]:
#Alfama

#Figure size
plt.rcParams['figure.figsize'] = [6, 6] 

#Subplots
fig, myax = plt.subplots()

#Plotting Alfama UGC data and defining markersize, color and transparency
flickr_alfama.plot(ax=myax,markersize=10, color='mediumpurple', alpha=0.6)
airbnb_alfama.plot(ax=myax,markersize=10, color='plum', alpha=0.6)
instagram_alfama.plot(ax=myax,markersize=10, color='firebrick', alpha=0.6)
twitter_alfama.plot(ax=myax,markersize=10, color='steelblue', alpha=0.6)

#Adding base-map with contextily
cx.add_basemap(myax, crs=twitter.crs.to_string(), source=cx.providers.CartoDB.Positron, zoom=15,attribution_size=0)

#Removing axis, adding scale bar and title
myax.axis("off")
myax.add_artist(ScaleBar(1, location='upper left'))
myax.set_title('Alfama')

#Defining personalized legends
twitter_leg = plt.scatter([],[], marker='o',color='steelblue', label='Twitter')
instagram_leg = plt.scatter([],[], marker='o',color='firebrick', label='Instagram')
flickr_leg = plt.scatter([],[], marker='o',color='mediumpurple', label='Flickr')
airbnb_leg = plt.scatter([],[], marker='o',color='plum', label='AirBnB')

#Adding legends
plt.legend(handles=[twitter_leg, instagram_leg, flickr_leg, airbnb_leg], loc='lower right')

In [ ]:
#Mouraria

#Figure size
plt.rcParams['figure.figsize'] = [6, 6] 

#Subplots
fig, myax = plt.subplots()

#Plotting Alfama UGC data and defining markersize, color and transparency
flickr_mouraria.plot(ax=myax,markersize=10, color='mediumpurple', alpha=0.6)
airbnb_mouraria.plot(ax=myax,markersize=10, color='plum', alpha=0.6)
instagram_mouraria.plot(ax=myax,markersize=10, color='firebrick', alpha=0.6)
twitter_mouraria.plot(ax=myax,markersize=10, color='steelblue', alpha=0.6)

#Adding base-map with contextily
cx.add_basemap(myax, crs=twitter.crs.to_string(), source=cx.providers.CartoDB.Positron, zoom=15,attribution_size=0)

#Removing axis, adding scale bar and title
myax.axis("off")
myax.add_artist(ScaleBar(1, location='upper left'))
myax.set_title('Mouraria')

#Defining personalized legends
twitter_leg = plt.scatter([],[], marker='o',color='steelblue', label='Twitter')
instagram_leg = plt.scatter([],[], marker='o',color='firebrick', label='Instagram')
flickr_leg = plt.scatter([],[], marker='o',color='mediumpurple', label='Flickr')
airbnb_leg = plt.scatter([],[], marker='o',color='plum', label='AirBnB')

#Adding legends
plt.legend(handles=[twitter_leg, instagram_leg, flickr_leg, airbnb_leg], loc='lower right')

In [ ]:
#Bairro Alto

#Figure size
plt.rcParams['figure.figsize'] = [6, 6] 

#Subplots
fig, myax = plt.subplots()

#Plotting Alfama UGC data and defining markersize, color and transparency
flickr_bairro.plot(ax=myax,markersize=10, color='mediumpurple', alpha=0.6)
airbnb_bairro.plot(ax=myax,markersize=10, color='plum', alpha=0.6)
instagram_bairro.plot(ax=myax,markersize=10, color='firebrick', alpha=0.6)
twitter_bairro.plot(ax=myax,markersize=10, color='steelblue', alpha=0.6)

#Adding base-map with contextily
cx.add_basemap(myax, crs=twitter.crs.to_string(), source=cx.providers.CartoDB.Positron, zoom=15,attribution_size=0)

#Removing axis, adding scale bar and title
myax.axis("off")
myax.add_artist(ScaleBar(1, location='upper left'))
myax.set_title('Bairro Alto')

#Defining personalized legends
twitter_leg = plt.scatter([],[], marker='o',color='steelblue', label='Twitter')
instagram_leg = plt.scatter([],[], marker='o',color='firebrick', label='Instagram')
flickr_leg = plt.scatter([],[], marker='o',color='mediumpurple', label='Flickr')
airbnb_leg = plt.scatter([],[], marker='o',color='plum', label='AirBnB')

#Adding legends
plt.legend(handles=[twitter_leg, instagram_leg, flickr_leg, airbnb_leg], loc='lower right')

### Creating shapes from geo-tagged activity using A-DBSCAN
#### Carrying out density-based clustering and retrieving alpha-shapes for each source and neighborhood

##### Source:
https://pysal.org/esda/notebooks/adbscan_berlin_example.html

We will need to set the EPS and the minimum samples parameters. 

### Alfama

In [ ]:
#Adding our geodataframes to a list
alfama_list = [twitter_alfama, instagram_alfama, flickr_alfama, airbnb_alfama]

#Now, we will perform a nearest neighbor analysis and obtain each distribution's median

# Create an empty list to store the results
eps_results = []

# Iterate over the geodataframes list for Alfama
for gdf in alfama_list:

    # Remove duplicated geometries
    gdf = gdf.drop_duplicates(subset='geometry')

    # Get the coordinates from the GeoDataFrame's geometry column
    coordinates = np.array(list(gdf.geometry.apply(lambda geom: (geom.x, geom.y))))

    # Compute the distance matrix
    dist_matrix = distance.cdist(coordinates, coordinates)

    # Flatten the distance matrix
    flattened_dist = dist_matrix.flatten()
    
    # Calculate the median
    medians = np.percentile(flattened_dist, 50)

    # Append the result to the list
    eps_results.append(medians)
    
# These are the distances we will use as our eps parameters for each clustering    
print(eps_results)

#### The following workflow applies the A-DBSCAN algorithm based on the distances we retrieved and set the minimum samples as half of our datasets
##### Running a sensitivity analysis is out of scope here, but feel free to change parameters

In [ ]:
#To apply the algorithm we need to get X and Y columns corresponding to long and lat values

#Iterating through the alfama datasets for each source and adding X and Y columns
for gdf in alfama_list:
    # Extract the longitude and latitude from the geometry column
    gdf['X'] = gdf.geometry.x.values
    gdf['Y'] = gdf.geometry.y.values
    
    
# Now let's apply the A-DBSCAN for each geodataframe using our eps_results distance
# and half of instances as our minimum samples to form a cluster

# Create an empty list to store the ADBSCAN models
adbs_list = []

# Iterate over the GeoDataFrames and eps values
for gdf, eps in zip(alfama_list, eps_results):
    
    #Remove duplicated geometries
    gdf = gdf.drop_duplicates(subset='geometry')
    
    # Calculate min_samples based on the current GeoDataFrame
    # Half of the count of instances
    min_samples = int(len(gdf)*0.50)

    # Create ADBSCAN object
    adbs = ADBSCAN(eps, min_samples, pct_exact=0.5, reps=50, keep_solus=True)
    
    # Set random seed for replication
    np.random.seed(1234)

    # Fit ADBSCAN on the current GeoDataFrame
    adbs.fit(gdf)
    
    # Append the fitted ADBSCAN model to the adbs_list
    adbs_list.append(adbs)
    
# Create an empty list to store the cluster boundaries
boundaries = []

# Iterate over the ADBSCAN models
for adbs, gdf in zip(adbs_list, alfama_list):
    
    # Get the cluster boundaries
    shape = get_cluster_boundary(adbs.votes["lbls"], gdf, ['X', 'Y'], crs=gdf.crs)

    # Append the cluster boundaries to the list
    boundaries.append(shape)
    
# List to store the transformed cluster boundaries to GeoDataFrames
boundary_gdfs = []

# Iterate over the boundaries
for boundary in boundaries:
    
    # Create a GeoDataFrame from the Polygon geometry
    gdf = gpd.GeoDataFrame(geometry=boundary, crs=twitter.crs)

    # Append the GeoDataFrame to the boundary_gdfs list
    boundary_gdfs.append(gdf)

#### Let's check the resulting shapes from our clustering for each source representing the Alfama neighborhood

In [ ]:
#Figure size
plt.rcParams['figure.figsize'] = [6, 6] 

#Subplots
fig, myax = plt.subplots()

#Plotting boundaries, defining edgecolors and linewidths
myax = boundary_gdfs[0].plot(ax=myax, edgecolor="steelblue", facecolor="none", linewidth=1.5)
boundary_gdfs[1].plot(ax=myax, edgecolor="firebrick", facecolor="none", linewidth=1.5)
boundary_gdfs[2].plot(ax=myax, edgecolor="mediumpurple", facecolor="none", linewidth=1.5)
boundary_gdfs[3].plot(ax=myax, edgecolor="plum", facecolor="none", linewidth=1.5)

#Adding base-map with contextily
cx.add_basemap(ax=myax, crs=twitter_alfama.crs.to_string(), attribution_size=0, zoom=15)

#Removing axis, adding title and scale bar
myax.axis("off");
myax.add_artist(ScaleBar(1, location='upper left'))
myax.set_title('A-DBSCAN shapes')


#Defining personalized legends
twitter = plt.scatter([],[], marker='o',color='steelblue', label='Twitter')
instagram = plt.scatter([],[], marker='o',color='firebrick', label='Instagram')
flickr = plt.scatter([],[], marker='o',color='mediumpurple', label='Flickr')
airbnb = plt.scatter([],[], marker='o',color='plum', label='AirBnb')

#Adding legends
plt.legend(handles=[twitter, instagram, flickr, airbnb], loc='lower right')

#### Now let's check the shapes with some color overlay to get a better perspective on the spatial overlap between sources

In [ ]:
#Figure size
plt.rcParams['figure.figsize'] = [6, 6] 

#Subplots
fig, myax = plt.subplots()

#Plotting the data with defined colors from the YlOrRd matplotlib color map and setting transparencies
myax = boundary_gdfs[1].plot(ax=myax, facecolor="#ffffcc", edgecolor="none", alpha=0.6)
boundary_gdfs[2].plot(ax=myax, facecolor="#febf5a", edgecolor="none", alpha=0.4)
boundary_gdfs[3].plot(ax=myax, facecolor="#f43d25", edgecolor="none", alpha=0.4)
boundary_gdfs[0].plot(ax=myax, facecolor="#800026", edgecolor="none", alpha=0.4)

#Adding base-map with contextily
cx.add_basemap(ax=myax, crs=twitter_alfama.crs.to_string(), attribution_size=0, zoom=15)

#Removing axis, adding scale bar and title
myax.axis("off");
myax.add_artist(ScaleBar(1, location='upper left'))
myax.set_title('A-DBSCAN shapes')

#Defining personalized legends
twitter = plt.scatter([],[], marker='o',color='#800026', label='Twitter')
instagram = plt.scatter([],[], marker='o',color='#ffffcc', label='Instagram')
flickr = plt.scatter([],[], marker='o',color='#febf5a', label='Flickr')
airbnb = plt.scatter([],[], marker='o',color='#f43d25', label='AirBnb')

#Adding legends
plt.legend(handles=[twitter, instagram, flickr, airbnb], loc='lower right')

### Let's check the survey responses for Alfama
##### We will remove invalid geometries, check the data and remove polygons that are objectively far off

In [ ]:
#First, let's check how many geometries are invalid
print(alfama_survey.geometry.is_valid.value_counts())

#Then, let's remove invalid geometries
alfama_survey = alfama_survey.loc[alfama_survey['geometry'].is_valid, :]

#### Now let's check the polygons

In [ ]:
#Figure size
plt.rcParams['figure.figsize'] = [6, 6] 

#Subplots
fig, myax = plt.subplots()

#Adding alfama survey polygons, defining color and transparency
alfama_survey.plot(ax=myax,markersize=20, color='indianred', alpha=0.3, edgecolor="white")

#Adding base-map with contextily
cx.add_basemap(myax, crs=alfama_survey.crs.to_string(), source=cx.providers.CartoDB.Positron, attribution_size=0)

#Removing axis, adding scale bar and title
myax.axis("off")
myax.add_artist(ScaleBar(1, location='upper left'))
myax.set_title('Alfama survey responses')

#### It seems there were some "errors" during data collection, let's remove these polygons by first obtaining the polygons' centroids, then performing a nearest neighbor analysis and obtaining their distribution

In [ ]:
#Obtaining centroids
centroids_alfama = gpd.GeoDataFrame(alfama_survey.geometry.centroid, geometry = alfama_survey.geometry.centroid)

#Obtaining the nearest neighbors within the same geodataframe
#Column foor storing the nearest geometry
centroids_alfama['nearest_geometry'] = '' 

#Iterating through index and rows of the geodataframe
for index, row in centroids_alfama.iterrows():
    point = row.geometry
    multipoint = centroids_alfama.drop(index, axis=0).geometry.unary_union
    queried_geom, nearest_geom = nearest_points(point, multipoint)
    centroids_alfama.loc[index, 'nearest_geometry'] = nearest_geom

In [ ]:
#Removing polygons that are too far from others

#obtaining the nearest distance distances for each centroid
nearest_centroids_alfama = gpd.GeoDataFrame(centroids_alfama.nearest_geometry, geometry = centroids_alfama.nearest_geometry)
nearest_centroids_alfama.set_crs(epsg = 32629, inplace = True)
nearest_centroids_alfama['distance'] = centroids_alfama.geometry.distance(nearest_centroids_alfama.geometry)

#threshold for removing - 90% percentile distribution
threshold = nearest_centroids_alfama['distance'].quantile(q=0.90)

#removing outlier centroids based on the threshold
alfama_centroids = nearest_centroids_alfama[nearest_centroids_alfama['distance'] <= threshold]

#removing correspondent outlier polygons
i1 = alfama_survey.index
i2 = alfama_centroids.index
alfama_survey_valid = alfama_survey[i1.isin(i2)]

#visualizing output polygons
plt.rcParams['figure.figsize'] = [6, 6] 
fig, myax = plt.subplots()
alfama_survey_valid.plot(ax=myax,markersize=20, color='indianred', alpha=0.3, edgecolor="white")
cx.add_basemap(myax, crs=alfama_survey_valid.crs.to_string(), attribution_size=0)

#setting the bounding boxes of visualization
myax.axis("off")
myax.add_artist(ScaleBar(1, location='upper left'))
myax.set_title('Alfama')
print(alfama_survey_valid.shape[0])

##### From 182 valid geometries, we ended up with 163 responses, now we will check their agreement or spatial overlap 
##### We first will define a function to retrieve all overlaps

In [ ]:
#function to retrieve all overlaps 
def count_overlapping_features(gdf):
    #generating all of the split pieces
    boundaries = gdf.geometry.exterior.unary_union
    new_polys = list(shapely.ops.polygonize(boundaries))
    new_gdf = gpd.GeoDataFrame(geometry=new_polys, crs=gdf.crs)
    new_gdf['id'] = range(len(new_gdf))

    #count overlapping by sjoin between pieces centroid and the input gdf 
    new_gdf_centroid = new_gdf.copy()
    new_gdf_centroid['geometry'] = new_gdf.centroid
    overlapcount = gpd.sjoin(new_gdf_centroid,gdf)
    overlapcount = overlapcount.groupby(['id'])['index_right'].count().rename('overlap_count').reset_index()
    out_gdf = pd.merge(new_gdf,overlapcount)
    return out_gdf

##### Checking the function

In [ ]:
#checking function
# Plot the GeoDataFrame
ax = count_overlapping_features(alfama_survey_valid).plot('overlap_count', cmap='viridis', edgecolor="none", legend=False)

# Get the minimum and maximum values for the colorbar
min_value = count_overlapping_features(alfama_survey_valid)['overlap_count'].min()
max_value = count_overlapping_features(alfama_survey_valid)['overlap_count'].max()

# Create a new colorbar
cbar = plt.colorbar(ax.get_children()[0], ax=ax, orientation='vertical', pad=0.1, aspect=40, shrink = 0.6)

# Set the colorbar ticks and tick labels
cbar.set_ticks([min_value, max_value])
cbar.set_ticklabels([min_value, max_value])

# Adjust the colorbar position
cbar.ax.yaxis.set_label_position('left')
cbar.ax.yaxis.set_ticks_position('left')

# Remove axis ticks
ax.set_xticks([])
ax.set_yticks([])

# Show the plot
plt.show()

##### We have up until 107 overlap geometries, now we will visualize it better and see in terms of percentages onto the basemap of Lisbon

In [ ]:
#Getting the overlap counts from the function
#overlap gdf
overlap_alfama_survey = count_overlapping_features(alfama_survey_valid)
count_percentage = (overlap_alfama_survey['overlap_count'] - overlap_alfama_survey['overlap_count'].min()) / (overlap_alfama_survey['overlap_count'].max() - overlap_alfama_survey['overlap_count'].min())
overlap_alfama_survey['count_percentage'] = count_percentage.round(decimals=2)*100

##### Let's take a closer look

In [ ]:
#Plotting in percentages - larger scale and removing small percentages (less than 20%)

#Subplots
fig, myax = plt.subplots()

#Plotting higher than 20% overlap, setting color map, legend and transparency
overlap_alfama_survey[overlap_alfama_survey['count_percentage'] > 20].plot('count_percentage', ax=myax, cmap='YlOrRd', alpha=0.5, edgecolor="none", legend=True, legend_kwds={'shrink': 0.45})

#Adding base-map with contextily
cx.add_basemap(myax, crs=alfama_survey_valid.crs.to_string(), source=cx.providers.CartoDB.Positron, attribution_size=0, zoom=15)

#Adding scale, title and removing axis
myax.set_title('Survey agreement (>20% overlap)')
myax.add_artist(ScaleBar(1, location='upper left'))
myax.axis("off")

### The last visualization will put side to side the agreement between participants and the agreement between sources

##### What can they tell us?

In [ ]:
#Create a figure with two subplots and set figure size
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

# getting the extent of the survey overlaps higher than 20% layer
# to set the same extent for both maps
xmin, ymin, xmax, ymax = overlap_alfama_survey[overlap_alfama_survey['count_percentage'] > 20].total_bounds

# First subplot - Spatial agreement between participants
plt.rcParams['figure.figsize'] = [8, 8]
ax1 = overlap_alfama_survey[overlap_alfama_survey['count_percentage'] > 20].plot('count_percentage', ax=ax1, cmap='YlOrRd', alpha=0.5, edgecolor="none", legend=True,legend_kwds={'shrink': 0.6})
cx.add_basemap(ax=ax1, crs=alfama_survey_valid.crs.to_string(), attribution_size=0, zoom=15)
ax1.axis("off")
ax1.set_title('Spatial agreement between participants')
ax1.add_artist(ScaleBar(1, location='upper left'))

# Second subplot - Spatial agreement between sources
plt.rcParams['figure.figsize'] = [8, 8] 
ax2 = boundary_gdfs[1].plot(ax=ax2, facecolor="#ffffcc", edgecolor="none", alpha=0.3)
boundary_gdfs[2].plot(ax=ax2, facecolor="#febf5a", edgecolor="none", alpha=0.3)
boundary_gdfs[3].plot(ax=ax2, facecolor="#f43d25", edgecolor="none", alpha=0.3)
boundary_gdfs[0].plot(ax=ax2, facecolor="#800026", edgecolor="none", alpha=0.3)
ax2.set_xlim(xmin-100, xmax+100)
ax2.set_ylim(ymin-100, ymax+100)
ax2.axis("off");
ax2.set_title('Spatial agreement between UGC sources')
ax2.add_artist(ScaleBar(1, location='upper left'))
cx.add_basemap(ax=ax2, crs=twitter_alfama.crs.to_string(), attribution_size=0, zoom=15)

#Defining personalized legends
twitter = plt.scatter([],[], marker='o',color='#800026', label='Twitter')
instagram = plt.scatter([],[], marker='o',color='#ffffcc', label='Instagram')
flickr = plt.scatter([],[], marker='o',color='#febf5a', label='Flickr')
airbnb = plt.scatter([],[], marker='o',color='#f43d25', label='AirBnb')

#Adding legends
plt.legend(handles=[twitter, instagram, flickr, airbnb], loc='lower right')

# Show the combined plot
plt.show()

### Now, based on above code, you should try to carry out the workflow with the other neighborhoods!

##### Remember to check parameters as for each source and neighborhood, representative sampling will change

<img src=https://c6.quickcachr.fotos.sapo.pt/i/o6e09e224/18830918_tQzwt.jpeg width="400">